### Testing Flow From Directory, Pre Processing

#### Table of contents
1. Overview
2. Image Data Generator (first attempt)
3. Image Data Generator (second attempt)
4. Other collected notes

#### 1. Overview
This notebook was used to try and set up an image data generator, where data would be streamed in from the hard drive containing the 100 gb of images, and process.  Part of the inestigation aimed to determine if there would be too much latency in pulling the images directly from the storage bucket into the model (spoiler alert - there would be).  This was also an early stage, where I thought that this would be runnable on local computers, but this quickly became a failed idea.

To begin with I pulled down a small sample of the data (2 images in test and train each for japan, china, france) to understand how long it takes to
 - pull them in
 - process
 
Once this worked, I'll look into pulling them in from GCP, though we may want to do this locally.

#### 2. Image Data Generator (first attempt)

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
os.getcwd()

'C:\\Users\\matfl\\Documents'

I will set up an image data generator, to flow from the directory, to make sure this works.

In [3]:
train_datagen = ImageDataGenerator()

In [4]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./MetClassifier/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


Found 6 images belonging to 3 classes.


In [5]:
test_datagen = ImageDataGenerator()

In [6]:
test_generator = test_datagen.flow_from_directory(
    directory=r"./MetClassifier/test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


Found 5 images belonging to 3 classes.


This works well enough, except that one of the images is a jfif.  I'm not doing any augmentation, or the image to array, preprocess images stuff that reshapes the tensor as needed.

Now, to see if this remotely works with gcp.  This points to the credentials to authenticate, and look at the bucket.

In [15]:
import os
from os import environ
from google.cloud import storage

credential_path = r"C:\Users\matfl\Documents\MSCA 31009 Project -24078705271d.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

storage_client = storage.Client()
client = storage_client
bucket = storage_client.get_bucket('met-image-bucket')

blob = bucket.blob('trn/')

Now to set up similar imagedatagenerator objects and flow from directories.

In [17]:
train_datagen_gcp = ImageDataGenerator()

In [20]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./trn/",
    target_size=(224, 224),
    classes = ['Japan', 'China', 'French', 'Italian', 'American'],
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


Found 0 images belonging to 5 classes.


So...yeah this didn't work.  Apparently *tensorflow* can read this in directly from GCP, but Keras can't.  So, let's try taht.  Links here: https://stackoverflow.com/questions/54736505/ideal-way-to-read-data-in-bucket-stored-batches-of-data-for-keras-ml-training-in

https://stackoverflow.com/questions/54523822/best-way-to-process-terabytes-of-data-on-gcloud-ml-engine-with-keras/54523859#54523859

https://medium.com/@moritzkrger/speeding-up-keras-with-tfrecord-datasets-5464f9836c36

https://www.tensorflow.org/guide/data

https://medium.com/@jianshi_94445/data-preparation-for-training-on-google-cloud-platform-using-tfrecords-d762574412f9

#### 3. Image Data Generator (attempt 2)

Here I tried to make a tensorflow dataset...long story short, this did not work well.

In [22]:
# Construct a TFRecordDataset
ds_train =  tf.data.TFRecordDataset('gs://met-image-bucket/test') # path to TFRecords on GCS
ds_train = ds_train.shuffle(1000).batch(32)


In [27]:
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model #sequential, to build the model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D #layers we need
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization #more types
from tensorflow.keras.preprocessing.image import load_img,img_to_array #processing images
#from tensorflow.keras.applications.imagenet_utils import preprocess_input #processing more
import tensorflow.keras.backend as K

In [31]:
tf.data.TFRecordDataset('gs://met-image-bucket/test')

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [70]:
import tensorflow as tf

# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


# load data in numpy
image,label = ReadFunctionToCreateYourDataAsNumpyArrays()


# create filewriter
writer = tf.python_io.TFRecordWriter(FILEPATH)


# Define the features of your tfrecord
feature = {'image':  _bytes_feature(tf.compat.as_bytes(image.tostring())),
           'label':  _int64_feature(int(label))}


# Serialize to string and write to file
example = tf.train.Example(features=tf.train.Features(feature=feature))
writer.write(example.SerializeToString())


NameError: name 'ReadFunctionToCreateYourDataAsNumpyArrays' is not defined

#### 4. Other random things I looked into

In [32]:
def _int64_feature(value):
  return tf.train.Feature(
            int64_list=tf.train.Int64List(value=value)
         )
def _floats_feature(value):
    return tf.train.Feature(
               float_list=tf.train.FloatList(value=value)
           )

def _bytes_feature(value):
  return tf.train.Feature(
              bytes_list=tf.train.BytesList(value=value)
         )


In [33]:
def load_image(path):
    img = cv2.imread(path)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img


I'll probably need to install cv2 into this environment. We'll also need to ensure that every image is the same size, but that will be once we figure out how to pull in the images.

In [47]:
from tensorflow.python.lib.io import file_io
filetest = file_io.FileIO('gs://met-image-bucket/trn/French/32157_1.jpg', mode='w+')

In [48]:
filetest

In [57]:
import gcsfs
#!pip install gcsfs
# https://gcsfs.readthedocs.io/en/latest/
import os
from os import environ
from google.cloud import storage

credential_path = r"C:\Users\matfl\Documents\MSCA 31009 Project -24078705271d.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path


In [68]:
fs = gcsfs.GCSFileSystem(project='MSCA 31009 Project', token=r"C:\Users\matfl\Documents\MSCA 31009 Project -24078705271d.json")
fs.ls('met-image-bucket/trn/')

['met-image-bucket/trn/American',
 'met-image-bucket/trn/China',
 'met-image-bucket/trn/French',
 'met-image-bucket/trn/Italian',
 'met-image-bucket/trn/Japan']

This...might...work

In [69]:
train_generator_gcp = train_datagen_gcp.flow_from_directory(
    directory=fs.ls('met-image-bucket/trn/'),
    target_size=(224, 224),
    classes = ['Japan', 'China', 'French', 'Italian', 'American'],
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


TypeError: expected str, bytes or os.PathLike object, not list